# Tutorial 3: Simple Reinforcement Learning

This tutorial aims to show you how to implement simple reinforcement learning agents into an ESPResSo simulation. After the tutorial, you will have learned:

* How to implement a reinforcement learning simulation in SwarmRL
* How to save a re-load trained agents
* Some tips for doing reinforcement learning with SwarmRL

## Imports

Once again, we must import some packages to get us through the tutorial.  We will try to keep the imports minimal and use longer calls during the tutorial so you know where a module or class comes from.

In [ ]:
# SwarmRL Imports
import swarmrl as srl

# ESPResSo Imports
import espressomd

# Linalg Imports
import numpy as np

# Neural Network Imports
import flax.linen as nn
import optax

# Unit Handling
import pint

# Plotting
import matplotlib.pyplot as plt

## Building the Simulation

Now, we need to create a simulation box and add colloids. We will add 40 colloids of two types to a 1000-micrometer box. We will also move to a slightly different interface for building simulations, namely one in which the system runner will be called from a function. The reason will become apparent in later tutorials, but it is good to start early.

In [ ]:
ureg = pint.UnitRegistry()  # Still define this outside.

system = espressomd.System(box_l=[1, 2, 3])  # This is just a dummy holder.
def get_system_runner(system):
    """
    Create a system runner.
    """
    md_params = srl.engine.espresso.MDParams(
            ureg=ureg,
            fluid_dyn_viscosity=ureg.Quantity(8.9e-4, "pascal * second"),
            WCA_epsilon=ureg.Quantity(293, "kelvin") * ureg.boltzmann_constant,
            temperature=ureg.Quantity(293, "kelvin"),
            box_length=ureg.Quantity(3 * [1000], "micrometer"),
            time_slice=ureg.Quantity(0.2, "second"),  # model timestep
            time_step=ureg.Quantity(0.02, "second") / 5,  # integrator timestep
            write_interval=ureg.Quantity(2, "second"),
        )
    system_runner = srl.engine.espresso.EspressoMD(
            md_params=md_params,
            n_dims=2,
            seed=np.random.randint(5453),  # seed for the simulation velocities
            out_folder="tutorial-2",
            write_chunk_size=1000,  # Used for dumping to the database.
            system=system,  # Add the pre-defined system.
        )
    
    # Add type 0 colloids to the simulation
    system_runner.add_colloids(
            n_colloids=20,  # Let's make 10 of them
            radius_colloid=ureg.Quantity(1.0, "micrometer"),
            random_placement_center=ureg.Quantity(
                np.array([500, 500, 0]), "micrometer"
            ),
            random_placement_radius=ureg.Quantity(60, "micrometer"),
            type_colloid=0,  # These are type 0
        )
    
    # Add type 1 colloids to the simulation
    system_runner.add_colloids(
            n_colloids=20,  # Let's make 10 of them
            radius_colloid=ureg.Quantity(1.0, "micrometer"),
            random_placement_center=ureg.Quantity(
                np.array([500, 500, 0]), "micrometer"
            ),
            random_placement_radius=ureg.Quantity(60, "micrometer"),
            type_colloid=1,  # These are type 1
        )
    
    return system_runner


## Creating the Reinforcement Learning Agent

Now, we can create an RL agent that can learn to perform chemotaxis. To do so, we need to define the following properties:

* Observable: What the agent sees in its environment.
* Task: What we want the agent to achieve.
* Actions: What the agent is capable of doing.
* Network: The neural networks used in the agent's brain.

In this tutorial, we will use chemical sensing as the observable. This means the agent can sense some chemical or field changes around it when moving. As a task, we will ask the agent to maximise these changes to move closer to the source of the chemical. The actions will be simple: translation along its directional axis, rotation clockwise, counterclockwise, or doing nothing.

### Decay Function
For both the task and the observable, we need to define how the concentration of our field falls off from its source. Let's use a simple linear function.

In [ ]:
def decay_fn(distance: np.ndarray):
    return 1.0 - distance

### Observable
Now, we can define the observable. The agent will receive this as input to the neural network when it makes decisions.

In [ ]:
observable = srl.observables.ConcentrationField(
        source=np.array([500.0, 500.0, 0.0]),  # Source is the middle of the box
        decay_fn=decay_fn,
        scale_factor=1000,  # Scales the reward which might otherwise be very small.
        box_length=np.array([1000.0, 1000.0, 1000]),  # Normalizes distances.
        particle_type=0,  # Only acts on type 0 colloids.
    )

### Task
Let's now give our colloids something to do.

In [ ]:
task = srl.tasks.searching.GradientSensing(
        source=np.array([500.0, 500.0, 0.0]),
        decay_function=decay_fn,
        reward_scale_factor=1000,
        box_length=np.array([1000.0, 1000.0, 1000]),
    )

### Actions
Now, some actions. Note that the keys in the following dictionary are just for your benefit; the agent cares about the position of the action, not its name.

In [ ]:
actions = {
    "RotateClockwise": srl.actions.Action(torque=np.array([0.0, 0.0, 10.0])),
    "Translate": srl.actions.Action(force=10.0),
    "RotateCounterClockwise": srl.actions.Action(torque=np.array([0.0, 0.0, 10.0])),
    "DoNothing": srl.actions.Action(),
}

### Brain
Now, let's give our agent a brain. The architecture of SwarmRL is such that we take a single neural network module that can have as many networks inside it as desired. In this case, we will use a shared architecture for the neural network. This means that the actor-networkand critic networks will share layers.

We should make a point here that we are leaving out some optional parameters that can be added to the network: an exploration policy and a sampling strategy. These will be discussed in later tutorials, which go into more detail about what one can change about the learning process.

In [ ]:
class ActorCriticNet(nn.Module):
    """A simple dense neural newtork."""

    @nn.compact
    def __call__(self, x):
        x = nn.Dense(features=12)(x)  # Shared layer,
        x = nn.relu(x)
        y = nn.Dense(features=1)(x)
        x = nn.Dense(features=4)(x)
        return x, y

In [ ]:
# Now convert this into a SwarmRL network.
network = srl.networks.FlaxModel(
    flax_model=ActorCriticNet(),
    optimizer=optax.adam(learning_rate=0.001),
    input_shape=(1,)  # Ipnut to the network is a single number.
)

### Agent
We can finally define an agent that will be controlled by this neural network and trained during the simulations. Today, we are using the Actor-critical agent, which, as the name suggests, is prepared using actor-critic reinforcement learning.

In [ ]:
agent = srl.agents.ActorCriticAgent(
    particle_type=0,
    network=network,
    task=task,
    observable=observable,
    actions=actions,
)

### Trainer
Now, we need to decide how to train the network. There are a few ways to do this:
* Episodic: Run the agents for a set amount before resetting the simulation and updating the networks.
* Semi-episodic: Update the agents during the simulation but reset it occasionally.
* Continuous: Start the simulation and let it run while the agents try to learn.

In this tutorial, we will try out both approaches together and as a demonstration of a realistic training procedure.

In [ ]:
continuous_trainer = srl.trainers.ContinuousTrainer(
    [agent],
)

### Training
Now, we can train the neural network. To do so, we run 100 episodes of length 50. This 50 means that the network is called for new actions 50 times inside the simulation before all rewards are collected and used to update the network. After training, we can plot the rewards to see how the agents are doing.

In [ ]:
rewards = []

system_runner = get_system_runner(system)
rewards.append(continuous_trainer.perform_rl_training(
    system_runner=system_runner,
    n_episodes=500,
    episode_length=10,
))

In [ ]:
plt.plot(np.concatenate((rewards))
plt.xlabel("Episode")
plt.ylabel("Reward")
plt.show()

As you can see, the agents haven't started to learn. But training is still early on. Try increasing the number of episodes and see if you notice when the agents begin to train.

Let's say you have trained for a while with continuous training, but the agents are running away from their target or are unable to find good rewards. Let's try to train them with the episodic trainer for a while and see if it makes a difference.

In [ ]:
episodic_trainer = srl.trainers.EpisodicTrainer(
    [agent],
)

In [ ]:
rewards.append(episodic_trainer.perform_rl_training(
    get_engine=get_system_runner,
    n_episodes=500,
    system=system,
    episode_length=50,  # We should give them more time to reach the target.
    reset_frequency=1  # Increase this for semi-episodic training
))

In [ ]:
plt.plot(np.concatenate(rewards))
plt.xlabel("Episode")
plt.ylabel("Reward")
plt.show()

Maybe they're doing a bit better, but they take a while to reach the centre, and I am unsure if they receive enough feedback. This is an excellent point to test semi-episodic training to get the best of both worlds.

In [ ]:
semi_episodic_trainer = srl.trainers.EpisodicTrainer(
    [agent],
)

In [ ]:
rewards.append(semi_episodic_trainer.perform_rl_training(
    get_engine=get_system_runner,
    n_episodes=500,
    system=system,
    episode_length=10,
    reset_frequency=20  # Reset the environment after 20 episodes.
))

In [ ]:
plt.plot(np.concatenate(rewards))
plt.xlabel("Episode")
plt.ylabel("Reward")
plt.show()

Unfortunately, I can't promise that any of this will work. RL sometimes converges quickly, and other times not at all. However, with enough training, this should find its source.

### Saving the agent
Training is one thing, but we want to be able to deploy this agent into simulations without training or even continue training this agent later on. To do so, we can save the state of the agent networks and optimizer either through the trainer or the agent.

In [ ]:
# trainer.export_models(directory="Models") #<-- Alternative method.
agent.save_agent("Models")

If we wanted to, we could load this agent up again and keep training.

In [ ]:
# trainer.restore_models(directory="Models")

However, I can also create the agent again, load in the state of it, and deploy it directly in my simulation.

In [ ]:
agent = srl.agents.ActorCriticAgent(
    particle_type=0,
    network=network,
    task=task,
    observable=observable,
    actions=actions,
)
agent.restore_agent("Models")

Finally, I will create a force function using the agent and run an ESPResSo simulation with my newly trained agents. Keep in mind no training will occur now.

In [ ]:
force_fn = srl.force_functions.ForceFunction({"0": agent})
system_runner = get_system_runner(system)
system_runner.integrate(1000, force_fn)